周内效应 一

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
asset = 'jrj.510310'
# baselineasset = 'jrj.510310'
baselineasset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = trdb2py.str2timestamp('2019-01-01', '%Y-%m-%d')

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = trdb2py.str2timestamp('2020-01-01', '%Y-%m-%d')

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [ ]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(baselineasset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(baselineasset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300指数',
)  

pnlBaseline1 = trdb2py.simTrading(trdb2cfg, p0)
# trdb2py.showPNL(pnlBaseline1, toImg=isStaticImg, width=width, height=height)

大家好，我是格子衫小C，提到格子衫，大家第一想到的应该就是理工男，我并不想颠覆大家的这个认知，我确实是理工男。

我记得有句老话，人一生一定要交的朋友，有医生、律师这些的，这些都很对，我只想补充一句，在网络时代，还应该交个理工男朋友，哪怕只是一个工具人朋友都好，当然，最好就是我了。

我能帮你做些什么呢？

首先，我会用科学的方法来帮你分析问题，但不会用特定数据来引导你做出结论。

现在，我们先来看看，各种资产的周内效应。

所谓周内效应，就是资产价格，在一周内，有没有一个明显的可利用的规律。

简单点说，有没有可能周一买周二卖，就能赚钱的。

先说结论，有的，而且有个很简单的策略，15年40倍的回报率（国内开放式基金里，最大回报率的差不多也就20多倍）

这个策略，平均年化能到200%，夏普超过10。

In [ ]:
# asset = 'jqdata.000300_XSHG|1d'
# asset = 'jqdata.000905_XSHG|1d'
asset1 = 'jqdata.000932_XSHG|1d'

s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset1),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday2',
    vals=[4, calcweekday2val2(4, 4)],
)

buy1 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.{}'.format(29)],
)

buy2 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday2',
    vals=[1, calcweekday2val2(1, 4)],
    group=1,
)

buy3 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ema.29'],
    group=1,    
)

sell0 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday',
    vals=[3],
)

sell1 = trdb2py.trading2_pb2.CtrlCondition(
    name='ctrlconditionid',
    vals=[1],
    strVals=['buy'],
)

sell2 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday',
    vals=[5],
    group=1,     
)

sell3 = trdb2py.trading2_pb2.CtrlCondition(
    name='ctrlconditionid',
    vals=[2],
    strVals=['buy'],
    group=1,     
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0, buy1, buy2, buy3])
s0.sell.extend([sell0, sell1, sell2, sell3])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsSell.CopyFrom(paramssell) 
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset1)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='周内效应混合策略',
)  

pnlm = trdb2py.simTrading(trdb2cfg, p0)

trdb2py.showPNLs([pnlm, pnlBaseline1], toImg=isStaticImg, width=width, height=height)

我们先找沪深300指数来看看吧。

首先看看沪深300的基准表现，就是在开始时间就直接买入，然后一直持有，看具体的收益率。

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

那么策略基准线大概就是这样了，7年多的时间2.2倍。

接下来，先看看最简单的情况，就是特定周几买入，第二天就卖出，只持有1天。

In [ ]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[lststart[i]],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天'.format(lsttitle[i], 1),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

这里还需要注意一点的是，我们用的策略是类似国内的基金策略，周一买周二卖，是用周一的结束价格买入，周二的结束价格卖出。

所以，周三买入周四卖出，其实表示的是周四的价格变化。

如果看曲线图不是很清楚的话，我们列表看看

In [ ]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

我看可以看到，周一周五会明显好过周二周三周四。

虽然盈利都不如基线，但由于波动率降低了。

特别是周一和周五，最大回撤都降低了50%以上，所以夏普都要高于基线。

上面的策略比较简单，就是周一买，周二卖，如果周二因为一些节假日，放假了，会持有到第一个可以交易的周二才卖。

为了严格的执行周内策略，我们换一下策略，改成如果周二不能卖，我们干脆周一就不买了，这样虽然会少一些交易次数，但更能反映出实际的结果。

In [ ]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lststart[i], calcweekday2val2(i + 1, 1)],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天v2'.format(lsttitle[i], 1),
    ))
 
lstpnl1t = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1t + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + lstpnl1t + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

可以看到，虽然有一些差别，但其实影响有好有坏，下面我们还是按v2版的策略来分析吧。

因为策略非常简单，我们来看看各种胜率，就是卖出价格高于买入价格的比例。

对于这种策略来说，如果没有周内效应，按道理胜率在50%左右。

In [ ]:
wrmt = trdb2py.buildPNLWinRateInYears(lstpnl1t)

wrmt[0][['title', 'total']]

我们可以看到，周一和周五的平均胜率接近60%了，周三的胜率在42%多，这应该算是有明显规律了。

接下来，我们看看年度胜率，确定一下，这个趋势是否有加强或减缓。

In [ ]:
# trdb2py.showBarWinRate4Month(lstpnl1t, valtype='abs', valoff=-0.5, toImg=isStaticImg, width=width, height=height)
# trdb2py.showBarWinRate4Month(lstpnl1t, toImg=isStaticImg, width=width, height=height)
trdb2py.showBarWinRateInYears(lstpnl1t, toImg=isStaticImg, width=width, height=height)

可以分别看看每一条数据，会发现，周五和周一还是明显要高于0.5的，而周二周三明显低于0.5。

还有个趋势，就是在广义的牛市年份里（2015年），几乎没有低于0.5的了，也就是说，虽然有周内效应，但牛市和熊市还是有差别的。

接下来，顺便看看月度胜率报表。

In [ ]:
trdb2py.showBarWinRate4Month(lstpnl1t, toImg=isStaticImg, width=width, height=height)

前面最好的周五买入周一卖出，在4月胜率低到了30%，而最差的周三买入周四卖出，则是在11月胜率有了明显提升。

我们生成了4月的年度数据，来看看到底发生了什么。

In [ ]:
trdb2py.showBarWinRateInMonths(lstpnl1t, month=4, toImg=isStaticImg, width=width, height=height)

4月确实是个非常惨烈的月份，除了2015年和2020年外，周五买入周一卖出都是亏的。

但是，2015年和2020年都是牛市吧，我们如果简单的拿月份来跳过买入，是很不合理的，应该还有更好的方案。

再来看看11月。

In [ ]:
trdb2py.showBarWinRateInMonths(lstpnl1t, month=11, toImg=isStaticImg, width=width, height=height)

周三买入周四卖出这个策略，基本上就只在2017年有明显提升，2017年整年也算是上涨趋势，但事前不可能知道这点，所以按月分析暂时没啥有用的结果。

我们基本上能确定沪深300指数，有一定的周内效应，但月度效应不明显。

周四大概率是跌的（我们的策略里是周三买入周四卖出），周一、周二、周五大概率是涨的，现在的问题是，单独每天的效果没那么明显，还不如基线的回报高。

但如果能把周五、周一的增长叠加起来，是不是结果会更好一些呢。

干脆把持有1天到4天的情况，也就是从 周一买周二 卖，一直到 周一买周五卖，一起来看看

In [ ]:
lstparams = []

for day in range(1, 5):
    for i in range(0, 5):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday2',
            vals=[lststart[i], calcweekday2val2(i + 1, day)],
        )

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[trdb2py.nextWeekDay(lststart[i], day)],
        )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='{}买入持有{}天v2'.format(lsttitle[i], day),
        ))
 
lstpnl = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnl + [pnlBaseline])

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

上面是我们遍历的各种情况，5天买入，持有1-4天卖出，一共20种。

单看回报率，终于能眼睛一亮了，有能明显超过基线的了，而且，最好的情况，夏普提升了近3倍。

前面看胜率的时候，还发现牛市时和熊市时，其实周内效应的趋势有变化，这是不是也能利用一下呢？

我们可以拿最简单的均线来判断是否是牛市。

简单来说，就是遍历一下均线上下，周内买卖是否有差异。

均线简单的从5开始，到60吧，一共55条，每一条均线下，我们再穷举上面的20次运算，均线下，再穷举20次运算，一共是55X20X20，大概22000种情况。

In [ ]:
lstparams = []

for ema in range(5, 61):    
    for sdo in range(1, 5):
        for sd in range(1, 6):
            buy0 = trdb2py.trading2_pb2.CtrlCondition(
                name='weekday2',
                vals=[sd, calcweekday2val2(sd, sdo)],
            )

            buy1 = trdb2py.trading2_pb2.CtrlCondition(
                name='indicatorsp',
                operators=['up'],
                strVals=['ema.{}'.format(ema)],
            )
            
            sell0 = trdb2py.trading2_pb2.CtrlCondition(
                name='weekday',
                vals=[trdb2py.nextWeekDay(sd, sdo)],
            )

            sell1 = trdb2py.trading2_pb2.CtrlCondition(
                name='ctrlconditionid',
                vals=[1],
                strVals=['buy'],
            )            
            
            for edo in range(1, 5):
                for ed in range(1, 6):

                    buy2 = trdb2py.trading2_pb2.CtrlCondition(
                        name='weekday2',
                        vals=[ed, calcweekday2val2(ed, edo)],
                        group=1,
                    )

                    buy3 = trdb2py.trading2_pb2.CtrlCondition(
                        name='indicatorsp',
                        operators=['down'],
                        strVals=['ema.{}'.format(ema)],
                        group=1,    
                    )

                    sell2 = trdb2py.trading2_pb2.CtrlCondition(
                        name='weekday',
                        vals=[trdb2py.nextWeekDay(ed, edo)],
                        group=1,     
                    )

                    sell3 = trdb2py.trading2_pb2.CtrlCondition(
                        name='ctrlconditionid',
                        vals=[2],
                        strVals=['buy'],
                        group=1,     
                    )
            

                    s0 = trdb2py.trading2_pb2.Strategy(
                        name="normal",
                        asset=trdb2py.str2asset(asset),
                    )

                    s0.buy.extend([buy0, buy1, buy2, buy3])
                    s0.sell.extend([sell0, sell1, sell2, sell3])
                    s0.paramsBuy.CopyFrom(paramsbuy)
                    s0.paramsSell.CopyFrom(paramssell) 
                    s0.paramsInit.CopyFrom(paramsinit)        
                    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
                        assets=[trdb2py.str2asset(asset)],
                        startTs=tsStart,
                        endTs=tsEnd,
                        strategies=[s0],
                        title='ema{}up{}持有{}天down{}持有{}天'.format(ema, lsttitle[sd-1], sdo, lsttitle[ed-1], edo),
                    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=6)

trdb2py.showPNLs2(lstpnlmix, baseline=pnlBaseline, showNums=5, toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

我们可以看到，确实有了很大的提升，从3.5提升到了6。

因为我们现在是拿2013年到2020年的数据做的测试，我们把数据放大一些，看看完整的数据，特别是看看刚才的结论是否过拟合。

In [ ]:
asset = 'jqdata.000300_XSHG|1d'
# asset = 'jqdata.000905_XSHG|1d'
# asset = 'jqdata.000932_XSHG|1d'

s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday2',
    vals=[4, calcweekday2val2(4, 4)],
)

buy1 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.{}'.format(29)],
)

buy2 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday2',
    vals=[1, calcweekday2val2(1, 4)],
    group=1,
)

buy3 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ema.29'],
    group=1,    
)

sell0 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday',
    vals=[3],
)

sell1 = trdb2py.trading2_pb2.CtrlCondition(
    name='ctrlconditionid',
    vals=[1],
    strVals=['buy'],
)

sell2 = trdb2py.trading2_pb2.CtrlCondition(
    name='weekday',
    vals=[5],
    group=1,     
)

sell3 = trdb2py.trading2_pb2.CtrlCondition(
    name='ctrlconditionid',
    vals=[2],
    strVals=['buy'],
    group=1,     
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0, buy1, buy2, buy3])
s0.sell.extend([sell0, sell1, sell2, sell3])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsSell.CopyFrom(paramssell) 
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='周内效应混合策略',
)  

pnlm = trdb2py.simTrading(trdb2cfg, p0)

trdb2py.showPNLs([pnlm, pnlBaseline1], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport([pnlm, pnlBaseline1])

dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

2005年开始的话，这里的回报一下就到25了，就是简单的沪深300指数，我们就能做到这个回报。

周内效应，接下来还会有2篇，分别是横向扩展，就是看看其它指数或资产，是否也有周内效应，还有一篇实用策略篇，就是在现在大家能方便使用的渠道里，如何利用这个策略。

看到这里，我相信大家对格子衫小C也有一个基本的概念了，我是用数据和算力来进行分析，然后尽可能的模拟回测。

就好像均线，29日均线我估计很少人会用，均线算法其实有很多种（常用的好像就有7种，当然，在周内效应里我就随便选了一种，后面专门讲均线的时候，再来看其它的）。

然后尽可能的去解决过拟合的问题，就是回测数据和二次验证数据错开。